Данные https://huggingface.co/datasets/just-ai/jayguard-ner-benchmark 
получены из реальных сложных разговорных текстов, включая рабочие чаты, журналы поддержки клиентов и расшифровки разговорной речи.
Набор данных был создан Just AI для сравнения различных решений NER для анонимизации данных. Он специально предназначен для объектов, имеющих решающее значение для защиты персональных данных.

In [1]:
!pip install yargy -q
!pip install evaluate seqeval -q
# datasets  accelerate 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 82.2 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 43.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have

In [2]:
from google.colab import drive
import os
import pandas as pd
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import string
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.relations import gnc_relation
from yargy.predicates import gram, is_capitalized, type, caseless, eq, custom, normalized, dictionary, in_caseless
from yargy.tokenizer import Tokenizer
from yargy.pipelines import caseless_pipeline
from wordcloud import WordCloud
import numpy as np

from datasets import Dataset, DatasetDict
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import torch

2025-10-15 20:45:05.124906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760561105.302498      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760561105.354347      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [27]:
from datasets import load_dataset

ds = load_dataset("just-ai/jayguard-ner-benchmark")

In [28]:
train_data = ds['train']
train_pd = pd.DataFrame(train_data)
train_pd.head(2)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 850
    })
})

In [31]:
all_tags = set()
for tags in train_pd['ner_tags']:
    all_tags.update(tags)

all_tags = sorted(list(all_tags))
print(all_tags)

['B-FICT', 'B-GPE', 'B-PER', 'B-PERSON', 'B-PER_PUBLIC', 'B-PET', 'B-PUBLIC_PER', 'B-PUBLIC_PERSON', 'B-PUBLIC_PLACES', 'B-STREET_ADDRESS', 'B-THEO', 'O']


Есть похожие тэги, поэтому посмотрим на примеры текстов и различаются ли т.ги по смыслу или мы можем их объединить в один класс

In [32]:
tag_examples = {}

for tag in all_tags:
    for idx in range(len(train_pd)):
        tokens = train_pd['tokens'].iloc[idx]
        tags_list = train_pd['ner_tags'].iloc[idx]

        if tag in tags_list:
            tag_index = tags_list.index(tag)
            token_with_tag = tokens[tag_index]

            tag_examples[tag] = {
                'token': token_with_tag,
                'full_text': ' '.join(tokens)
            }
            break

print("Примеры для тегов:")
for tag, example in tag_examples.items():
    print(f"\nТег: {tag}")
    print(f"Токен: '{example['token']}'")
    print(f"Полный текст: {example['full_text']}")


Примеры для тегов:

Тег: B-FICT
Токен: 'Щелкунчика'
Полный текст: В итогах викторины по рождественским традициям отмечены ключевые вопросы: происхождение Щелкунчика (Германия), традиционные подарки и символы МВК. Победители уведомлены согласно регламенту.

Тег: B-GPE
Токен: 'Москве'
Полный текст: Слушай, я в 2005 в Москве на Тверской 15 ставила брекеты, дорого, но без удаления зубов мудрости обошлось!

Тег: B-PER
Токен: 'асия'
Полный текст: <template>"я получил сообщение о кредите, но я, асия хемлина, ничего не запрашивал. возможно, произошла ошибка, или кто-то использовал мои данные. я не нуждаюсь в кредите и прошу отменить все заявки. дьячков также подтверждает, что не подавал никаких заявок на кредит. пожалуйста, проверьте информацию и сообщите мне о результатах."</template>

Тег: B-PERSON
Токен: 'Иванова'
Полный текст: 2024-06-15 09:30 - учитель Иванова Елена, школа №12, ул. Пушкина 10, применяет индивидуальный подход, исключая уровниловку

Тег: B-PER_PUBLIC
Токен: 'Доном'
Полный т

Дублирование: B-PER, B-PERSON, B-PER_PUBLIC, B-PUBLIC_PER, B-PUBLIC_PERSON
Поэтому почистим тэги, придем к стандартному BIO виду тэгов. У меня в данных тэги только B-, но по логиче часть из них соответствует тэгу I-. поэтому переименуем их и будем использовать RuBert 

In [33]:
def map_b_tags(tags_list):
    """Унифицирует B-теги к 5 основным типам."""
    
    mapping = {
        # Персоны
        'B-PER': 'B-PERSON', 'B-PERSON': 'B-PERSON', 
        'B-PER_PUBLIC': 'B-PERSON', 'B-PUBLIC_PER': 'B-PERSON', 
        'B-PUBLIC_PERSON': 'B-PERSON',
        
        # Места
        'B-GPE': 'B-LOC', 'B-STREET_ADDRESS': 'B-LOC', 
        'B-PUBLIC_PLACES': 'B-LOC', 
        
        # Другие
        'B-FICT': 'B-FICT', 
        'B-PET': 'B-PET', 
        'B-THEO': 'B-THEO',
        'O': 'O'
    }

    return [mapping.get(tag, 'O') for tag in tags_list]


train_pd['ner_tags'] = train_pd['ner_tags'].apply(map_b_tags)

In [34]:
def convert_to_bio_scheme(tags):
    """Преобразует в правильную B-I-O схему."""
    bio_tags = []
    
    for i, current_tag in enumerate(tags):
        if current_tag == 'O':
            bio_tags.append('O')
        elif current_tag.startswith('B-'):
            entity_type = current_tag[2:]
            
            # Проверяем, является ли этот токен продолжением предыдущей сущности
            if i > 0 and bio_tags[i-1].startswith(('B-', 'I-')) and bio_tags[i-1][2:] == entity_type:
                # Если предыдущий токен той же сущности - это I-
                bio_tags.append(f'I-{entity_type}')
            else:
                # Иначе - это начало новой сущности (B-)
                bio_tags.append(current_tag)
        else:
            bio_tags.append('O')
            
    return bio_tags

# Применяем B-I-O преобразование
train_pd['ner_tags'] = train_pd['ner_tags'].apply(convert_to_bio_scheme)

In [35]:
from sklearn.model_selection import train_test_split
from collections import defaultdict
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
from evaluate import load 
import torch


In [36]:
# деление на тест и трейн
tokens_list = train_pd['tokens'].tolist()
ner_tags_list = train_pd['ner_tags'].tolist()

stratify_labels = []
for tags in ner_tags_list:
    if tags and len(tags) > 0:
        first_tag = tags[0] if isinstance(tags[0], str) else 'O'
    else:
        first_tag = 'O'
    stratify_labels.append(first_tag)

train_idx, test_idx = train_test_split(
    range(len(tokens_list)),
    test_size=0.2,
    random_state=55,
    stratify=stratify_labels
)

splits = {
    'train': {
        'tokens': [tokens_list[i] for i in train_idx],
        'ner_tags': [ner_tags_list[i] for i in train_idx]
    },
    'test': {
        'tokens': [tokens_list[i] for i in test_idx],
        'ner_tags': [ner_tags_list[i] for i in test_idx]
    }
}

In [37]:
def label_mappings(dataset):
    """Создает полные маппинги"""
    b_tags = set()
    for tags in dataset['ner_tags']:
        for tag in tags:
            if tag.startswith('B-'):
                b_tags.add(tag)
    
    # полный набор тегов
    all_tags = ['O'] 
    
    # Добавляем B-теги и I-теги
    for b_tag in sorted(b_tags):
        entity_type = b_tag[2:]  # B-PERSON -> PERSON
        all_tags.append(b_tag)
        all_tags.append(f'I-{entity_type}')
    
    all_tags = sorted(set(all_tags))
    
    id_to_label = {i: tag for i, tag in enumerate(all_tags)}
    label_to_id = {tag: i for i, tag in enumerate(all_tags)}

    for i, tag in id_to_label.items():
        print(f"  {i}: {tag}")
    
    return id_to_label, label_to_id

id_to_label, label_to_id = label_mappings(splits['train'])
num_labels = len(id_to_label)

  0: B-FICT
  1: B-LOC
  2: B-PERSON
  3: B-PET
  4: B-THEO
  5: I-FICT
  6: I-LOC
  7: I-PERSON
  8: I-PET
  9: I-THEO
  10: O


In [38]:
# MODEL_NAME = '/kaggle/input/deeppavlov-rubertbasecased'
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
# токенизация
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    trust_remote_code=True
)

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels, 
    id2label=id_to_label,
    label2id=label_to_id,
    trust_remote_code=True
)

def tokenize_and_align_labels_safe(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        max_length=512
    )
    labels = []

    for i, ner_tags in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Спец токены = -100
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # Первый токен слова - берем оригинальную метку
                original_label = ner_tags[word_idx]
                label_ids.append(label_to_id[original_label])
            else:
                # Последующие токены того же слова
                original_label = ner_tags[word_idx]
                
                if original_label.startswith("B-"):
                    # Преобразуем B- в I- для subwords
                    entity_type = original_label[2:]
                    i_label = f"I-{entity_type}"
                    
                    # Проверяем что I-метка существует в словаре
                    if i_label in label_to_id:
                        label_ids.append(label_to_id[i_label])
                    else:
                        # Если I-метки нет, используем оригинальную B-метку
                        label_ids.append(label_to_id[original_label])
                else:
                    # I- или O метки остаются как есть
                    label_ids.append(label_to_id[original_label])
            
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


Some weights of BertForTokenClassification were not initialized from the model checkpoint at /kaggle/input/deeppavlov-rubertbasecased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
splits_hf = DatasetDict({
    'train': Dataset.from_dict(splits['train']),
    'test': Dataset.from_dict(splits['test'])
})

tokenized_datasets = splits_hf.map(
    tokenize_and_align_labels_safe,
    batched=True,
    batch_size=8
)


del splits_hf
import gc; gc.collect()

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

243

### Дообучение модели на наших данных

In [40]:
training_args = TrainingArguments(
    output_dir="./rubert_ner_results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
    fp16=True,
    gradient_checkpointing=True
)

# для выравнивания данных
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True
)


In [41]:
metric = load("seqeval")  
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    # Возвращаем основные метрики
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#  Раскомментировать для тренировки модели и сохранения
trainer.train()

# trainer.save_model("./bert-ner-final-1")
# tokenizer.save_pretrained("./bert-ner-final-1")


/tmp/ipykernel_37/1651587683.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.310700,0.087964,0.224490,0.175299,0.196868,0.972101
2,0.071500,0.067362,0.342105,0.362550,0.352031,0.978329
3,0.051800,0.066253,0.349315,0.406375,0.375691,0.978076


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=255, training_loss=0.12228511712130379, metrics={'train_runtime': 152.458, 'train_samples_per_second': 13.381, 'train_steps_per_second': 1.673, 'total_flos': 495895853280432.0, 'train_loss': 0.12228511712130379, 'epoch': 3.0})

In [43]:
# trainer.save_model("./bert-ner-final")
# tokenizer.save_pretrained("./bert-ner-final")
# import shutil

# shutil.make_archive("bert-ner-model", 'zip', "./bert-ner-final")

('./bert-ner-final/tokenizer_config.json',
 './bert-ner-final/special_tokens_map.json',
 './bert-ner-final/vocab.txt',
 './bert-ner-final/added_tokens.json',
 './bert-ner-final/tokenizer.json')

In [68]:
def detail_metrics(trainer, dataset):

    predictions, labels, _ = trainer.predict(dataset)
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]
    true_labels = [[id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]


    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    return results, true_predictions, true_labels


detailed_results, true_predictions, true_labels = detail_metrics(trainer, tokenized_datasets["test"])

def create_metrics_table(detailed_results): 
    metrics_data = []

    for entity_type in detailed_results:
        if entity_type not in ['overall_precision', 'overall_recall', 'overall_f1', 'overall_accuracy']:
            metrics = detailed_results[entity_type]
            metrics_data.append({
                'Entity Type': entity_type,
                'Precision': f"{metrics['precision']:.1%}",
                'Recall': f"{metrics['recall']:.1%}",
                'F1-Score': f"{metrics['f1']:.1%}",
                'Support': metrics['number']
            })
    
    metrics_data.append({
        'Entity Type': 'OVERALL',
        'Precision': f"{detailed_results['overall_precision']:.1%}",
        'Recall': f"{detailed_results['overall_recall']:.1%}",
        'F1-Score': f"{detailed_results['overall_f1']:.1%}",
        'Support': ''
    })
    
    return pd.DataFrame(metrics_data)


metrics_df = create_metrics_table(detailed_results)
print(metrics_df.to_string(index=False))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Entity Type Precision Recall F1-Score Support
       FICT      0.0%   0.0%     0.0%       1
        LOC     70.1%  70.9%    70.5%      86
     PERSON     20.0%  27.2%    23.0%     151
        PET      0.0%   0.0%     0.0%       9
       THEO      0.0%   0.0%     0.0%       4
    OVERALL     34.9%  40.6%    37.6%        


Наиболее успешно определяется LOC, сбалансированы точность и полнота
Низкое качество для Person при наибольшем количестве примеров. 
Плохо определяются FICT, PET, полностью не распознаются эти категории, это происходит из-за недостаточных данных для обучения и дисбалансе классов

Возможно, что person, fict, pet определяются плохо, т.к. похожи и сущности из FICT, PET некорректно определяются как PERSON. Например, "Шарик" это PET или PERSON, "Мастер" → FICT или PERSON? 

### Загрузка модели без обучения для тестирования

In [45]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_path = "bert-ner-final"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./temp_test",
        per_device_eval_batch_size=16,
        report_to="none",
        no_cuda=True, 
    ),
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

detailed_results = detail_metrics(trainer, tokenized_datasets["test"])
metrics_df = create_metrics_table(detailed_results)
print(metrics_df.to_string(index=False))

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Entity Type Precision Recall F1-Score Support
       FICT      0.0%   0.0%     0.0%       1
        LOC     70.1%  70.9%    70.5%      86
     PERSON     20.0%  27.2%    23.0%     151
        PET      0.0%   0.0%     0.0%       9
       THEO      0.0%   0.0%     0.0%       4
    OVERALL     34.9%  40.6%    37.6%        


In [49]:
# metrics_df

## Добавление синтетически словарей, чтоб клички и персонажи лучше определялись

In [50]:
fict_names = [
    # Русская классика
    "Евгений Онегин", "Алексей Карамазов", "Родион Раскольников", "Пьер Безухов",
    "Наташа Ростова", "Анна Каренина", "Григорий Мелехов", "Мастер", "Маргарита",
    "Обломов", "Печорин", "Чичиков", "Базаров", "Шариков", "Собакевич", "Ноздрёв",
    "Манилов", "Платон Каратаев", "Андрей Болконский", "Лев Мышкин", "Софья Мармеладова",
    
    # Советская литература
    "Остап Бендер", "Киса Воробьянинов", "Шурик", "Электроник", "Буратино", "Мальвина",
    "Артемон", "Карабас Барабас", "Дядя Степа", "Федор", "Шарик", "Матроскин",
    "Пеппи Длинныйчулок", "Карлсон", "Малыш", "Винни Пух", "Пятачок", "Иа",
    
    # Зарубежная литература
    "Гарри Поттер", "Гермиона Грейнджер", "Рон Уизли", "Дамблдор", "Волдеморт",
    "Фродо Бэггинс", "Бильбо Бэггинс", "Гендальф", "Арагорн", "Голлум", "Леголас",
    "Дарт Вейдер", "Люк Скайуокер", "Хан Соло", "Шерлок Холмс", "Доктор Ватсон",
    "Мориарти", "Эркюль Пуаро", "Джеймс Бонд", "Гарри Гудини", "Лисбет Саландер",
    
    # Современная фантастика
    "Геральт из Ривии", "Йеннифэр", "Цири", "Тирион Ланнистер", "Джон Сноу",
    "Дейенерис Таргариен", "Арья Старк", "Санса Старк", "Хауленд Рид", "Бран Старк",
    "Перси Джексон", "Анабет Чейз", "Гарри Дрезден", "Конан", "Элрик",
    
    # Аниме и игры
    "Наруто Узумаки", "Саске Учиха", "Гоку", "Сэйлор Мун", "Эдвард Элрик",
    "Спайк Шпигель", "Лайт Ягами", "Эрен Йегер", "Линк", "Марио", "Соник",
    "Лара Крофт", "Гордон Фримен", "Мастер Чиф", "Кратос", "Эцио"
]

fict_contexts = [
    # Литературные контексты
    "герой романа", "персонаж книги", "в литературном произведении", "действующее лицо повести",
    "главный герой цикла", "литературный образ", "персонаж классики", "герой эпопеи",
    "в знаменитом произведении", "персонаж саги", "герой трилогии", "в бестселлере",
    
    # Жанровые контексты
    "фантастический персонаж", "герой фэнтези", "персонаж научной фантастики", "в детективе",
    "герой приключенческого романа", "персонаж мистики", "в антиутопии", "герой утопии",
    "персонаж киберпанка", "герой стимпанка", "в постапокалипсисе", "герой хоррора",
    
    # Медийные контексты
    "персонаж фильма", "герой сериала", "в киноэпопее", "персонаж аниме",
    "герой видеоигры", "в комиксах", "персонаж манги", "герой графического романа",
    "в телевизионном шоу", "персонаж веб-сериала", "герой анимации",
    
    # Характеристики
    "вымышленный персонаж", "придуманный герой", "художественный образ", "созданный автором",
    "плод фантазии", "литературное творение", "собирательный образ", "архетипический герой",
    "культовый персонаж", "знаковый образ", "узнаваемый герой"
]

fict_actions = [
    "появляется в кульминационной сцене", "противостоит главному злодею", "спасает мир от разрушения",
    "принимает судьбоносное решение", "проходит через испытания", "меняется на протяжении сюжета",
    "обучается магическим искусствам", "путешествует между мирами", "сражается с темными силами",
    "раскрывает древнюю тайну", "находит магический артефакт", "встречает верных союзников",
    "преодолевает внутренние демоны", "становится легендой", "меняет ход истории",
    "бросает вызов системе", "создает нечто великое", "теряет близких", "обретает любовь"
]

In [51]:
pet_names = [
    # Собаки мужские
    "Шарик", "Рекс", "Мухтар", "Полкан", "Дружок", "Барбос", "Тузик", "Лорд",
    "Цезарь", "Арес", "Зевс", "Барон", "Граф", "Корсар", "Пират", "Вулкан",
    "Бим", "Джек", "Рич", "Чарли", "Бадди", "Макс", "Рокки", "Бруно", "Оскар",
    "Арчи", "Тедди", "Ватсон", "Спайк", "Блейд", "Гром", "Вихрь", "Сокол",
    
    # Собаки женские
    "Лада", "Джесси", "Линда", "Сара", "Молли", "Люси", "Дейзи", "Мэгги",
    "Софи", "Луна", "Белла", "Рокси", "Ника", "Ирма", "Эльза", "Хлоя",
    "Жужа", "Искра", "Звезда", "Бусинка", "Пушинка", "Стрелка", "Лайма",
    
    # Кошки мужские
    "Васька", "Барсик", "Мурзик", "Рыжик", "Кузя", "Василий", "Леопольд",
    "Гарфилд", "Том", "Зорро", "Маркиз", "Филя", "Симба", "Амур", "Базилио",
    "Персик", "Сэм", "Оливер", "Симон", "Лео", "Тигра", "Спайк", "Феликс",
    
    # Кошки женские
    "Мурка", "Пушок", "Снежок", "Сима", "Багира", "Дымка", "Марта", "Муся",
    "Лиза", "Соня", "Ася", "Клеопатра", "Изида", "Луна", "Несси", "Фрекен",
    "Бьянка", "Джесси", "Мия", "Юми", "Сакура", "Химе", "Юки",
    
    # Универсальные/экзотические
    "Кеша", "Гоша", "Арчи", "Ричи", "Пушок", "Компот", "Боня", "Феня",
    "Цыпа", "Хома", "Пончик", "Бусинка", "Пушинка", "Злата", "Ириска",
    "Кнопка", "Сеня", "Фунтик", "Цветик", "Сема", "Нюша", "Крош"
]

pet_types = [
    "собака", "кот", "кошка", "пёс", "котёнок", "щенок", "попугай", "хомяк",
    "крыса", "морская свинка", "кролик", "шиншилла", "канарейка", "ворон",
    "грызун", "питомец", "любимец", "четвероногий друг", "пушистик", "зверёк"
]

pet_actions = [
    "играет с мячиком", "спит на диване", "ест свой корм", "бегает по двору",
    "лает на почтальона", "мяукает у двери", "грызёт свою игрушку", "прячется в коробке",
    "носится по квартире", "мурлычет на коленях", "охраняет дом", "ловит мышей",
    "учит команды", "гуляет в парке", "плавает в речке", "копает ямку",
    "греется на солнышке", "просит вкусняшку", "встречает с работы", "будит утром"
]

pet_contexts = [
    "домашний любимец", "верный друг", "член семьи", "пушистый компаньон",
    "моя радость", "наше сокровище", "домашний питомец", "любимый зверёк",
    "четвероногий друг", "пушистый член семьи", "верный спутник", "моя отрада"
]

In [54]:
train_pd

,tokens,ner_tags
0,"[Слушай,, я, в, 2005, в, Москве, на, Тверской,...","[O, O, O, O, O, B-LOC, O, B-LOC, I-LOC, O, O, ..."
1,"[Уважаемая, госпожа, Смирнова,, подтверждаем,,...","[O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, I-LO..."
2,"[Пациентка, прошла, ортодонтическое, лечение, ...","[O, O, O, O, O, O, O, O, B-LOC, I-LOC, I-LOC, ..."
3,"[2005-09-12, установка, брекетов,, Москва,, ул...","[O, O, O, O, B-LOC, I-LOC, O, O, O, O, O, O, O]"
4,"[привет, я, ставила, брекеты, на, тверской, 15...","[O, O, O, O, O, B-LOC, I-LOC, O, O, O, O, O, O..."
...,...,...
845,"[LAST_APP_RESPONSE:, Рекомендую, уведомить, Ал...","[O, O, O, B-PERSON, I-PERSON, O, O, O, O, O, O..."
846,"[LAST_APP_RESPONSE:, Рекомендую, уведомить, До...","[O, O, O, B-PERSON, I-PERSON, O, O, O, O, O, O..."
847,"[LAST_USER_REQUEST:, Перефразируй, -, Картогра...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
848,"[Диалог:, Обсуждение, подарка., Вопрос, диалог...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [55]:
# import random
# from transformers import TrainingArguments, Trainer
# from transformers import DataCollatorForTokenClassification


# def augment_dataset(dataset, num_fict=150, num_pet=100):
#     """Аугментация датасета с синтетическими примерами"""
    
#     texts = train_pd['tokens']
#     labels = train_pd['ner_tags']
    
#     print(f"До аугментации: {len(texts)} примеров")
    
#     augmented_texts = texts.copy()
#     augmented_labels = labels.copy()
    
#     # Генерация FICT примеров
#     for _ in range(num_fict):
#         name = random.choice(fict_names)
#         context = random.choice(fict_contexts)
#         action = random.choice(fict_actions)
        
#         templates = [
#             f"{name} - {context}.",
#             f"{context.capitalize()} {name} {action}.",
#             f"В известном произведении появляется {name}.",
#             f"{name} становится центральным персонажем.",
#             f"Образ {name} запомнился многим читателям."
#         ]
        
#         text = random.choice(templates)
#         tokens = text.split()
        
#         # Создаем разметку
#         text_labels = ['O'] * len(tokens)
#         name_tokens = name.split()
        
#         # Размечаем имя персонажа
#         for i, token in enumerate(tokens):
#             if token == name_tokens[0]:
#                 text_labels[i] = 'B-FICT'
#                 for j in range(1, len(name_tokens)):
#                     if i + j < len(tokens) and tokens[i + j] == name_tokens[j]:
#                         text_labels[i + j] = 'I-FICT'
        
#         augmented_texts.append(text)
#         augmented_labels.append(text_labels)
    
#     # Генерация PET примеров
#     for _ in range(num_pet):
#         name = random.choice(pet_names)
#         pet_type = random.choice(pet_types)
#         action = random.choice(pet_actions)
#         context = random.choice(pet_contexts)
        
#         templates = [
#             f"Мой {pet_type} {name} {action}.",
#             f"{name} - это {context}.",
#             f"Вчера {name} {action}.",
#             f"Наш {pet_type} по кличке {name} очень умный.",
#             f"{name} любит {action}."
#         ]
        
#         text = random.choice(templates)
#         tokens = text.split()
        
#         # Создаем разметку
#         text_labels = ['O'] * len(tokens)
        
#         # Размечаем кличку
#         for i, token in enumerate(tokens):
#             if token == name:
#                 text_labels[i] = 'B-PET'
        
#         augmented_texts.append(text)
#         augmented_labels.append(text_labels)
    
#     print(f"После аугментации: {len(augmented_texts)} примеров")
    
#     return {'text': augmented_texts, 'labels': augmented_labels}

# augmented_train_dataset = augment_dataset(
#     tokenized_datasets["train"], 
#     num_fict=150, 
#     num_pet=100
# )

До аугментации: 850 примеров


AttributeError: 'Series' object has no attribute 'append'

In [48]:
# training_args = TrainingArguments(
#     output_dir="./rubert_ner_results",
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     report_to="none",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     logging_steps=50,
#     fp16=True,
#     gradient_checkpointing=True
# )

# data_collator = DataCollatorForTokenClassification(
#     tokenizer=tokenizer,
#     padding=True
# )

# metric = load("seqeval")  

# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=2)

#     true_predictions = [
#         [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]

#     results = metric.compute(predictions=true_predictions, references=true_labels)
    
#     return {
#         "precision": results["overall_precision"],
#         "recall": results["overall_recall"],
#         "f1": results["overall_f1"],
#         "accuracy": results["overall_accuracy"],
#     }

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=augmented_train_dataset,  
#     eval_dataset=tokenized_datasets["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

# trainer.save_model("./bert-ner-final-model")
# tokenizer.save_pretrained("./bert-ner-final-model")

Применяем аугментацию...


NameError: name 's' is not defined

## Rule-based

In [60]:
from yargy.predicates import gram, dictionary, type as token_type
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline

In [61]:
# PER
person_name_rule = rule(
    gram('Name'),
    gram('Surn')
)

person_with_title_rule = rule(
    or_(
        dictionary(['президент', 'министр', 'депутат']),
        dictionary(['актер', 'режиссер', 'писатель']),
        dictionary(['ученый', 'профессор', 'доктор'])
    ),
    gram('Name'),
    gram('Surn')
)

final_person_rule = or_(person_name_rule, person_with_title_rule)
person_parser = Parser(final_person_rule)

# LOC
geo_with_type_rule = rule(
    or_(
        dictionary(['город', 'страна', 'деревня']),  # тип места
        dictionary(['столица', 'регион', 'область'])
    ),
    gram('Geox')
)
geo_simple_rule = rule(
    gram('Geox')
)

final_geo_rule = or_(geo_with_type_rule, geo_simple_rule)
geo_parser = Parser(final_geo_rule)

# ADRESS
address_rule = rule(
    or_(
        dictionary(['ул', 'улица', 'проспект']),
        dictionary(['д', 'дом'])
    ),
    gram('Surn'),
    token_type('INT').optional()
)

address_parser = Parser(address_rule)

# ORG
org_rule = rule(
    gram('Surn'),
    or_(
        dictionary(['ООО', 'ЗАО', 'АО', 'ИП']),
        dictionary(['компания', 'корпорация', 'банк'])
    )
)

org_parser = Parser(org_rule)


In [62]:
# MISC (FICT, PET, THEO)

def create_vocab(train_data, min_frequency=1):
    """Создаем словари на тренировочных данных"""

    entity_vocab = defaultdict(set)

    for tokens, tags in zip(train_data['tokens'], train_data['ner_tags']):
        current_entity = []
        current_type = None

        for token, tag in zip(tokens, tags):
            if tag.startswith('B-'):
                if current_entity and current_type:
                    entity_text = ' '.join(current_entity).lower()
                    entity_vocab[current_type].add(entity_text)

                current_entity = [token]
                current_type = tag[2:]

            else:
                if current_entity and current_type:
                    entity_text = ' '.join(current_entity).lower()
                    entity_vocab[current_type].add(entity_text)
                current_entity = []
                current_type = None

        if current_entity and current_type:
            entity_text = ' '.join(current_entity).lower()
            entity_vocab[current_type].add(entity_text)


    return entity_vocab

train_vocab = create_vocab(splits['train'])
# PET
pet_parser = Parser(rule(
    morph_pipeline(['кот', 'кошка', 'собака', 'пёс', 'щенок', 'питомец', 'животное', 'зверь']),
    gram('Name'),
    gram('Name').optional()
))

# FICT
fict_parser = Parser(rule(
    morph_pipeline(['сказка', 'персонаж', 'герой', 'фильм', 'книга', 'рассказ', 'повесть', 'роман']),
    gram('Name').repeatable(max=3)
))

# THEO
theo_parser = Parser(rule(
    morph_pipeline(['теория', 'эффект', 'закон', 'принцип', 'концепция', 'модель', 'гипотеза']),
    gram('Name').repeatable(max=3)
))


In [63]:
def extract_entities(text):
    entities = []
    predicted_spans = {}

    all_parsers = {
        'PERSON': person_parser,
        'LOC': geo_parser,
        'ADDRESS': address_parser,
        'ORG': org_parser,
        'PET': pet_parser,
        'FICT' : fict_parser,
        'THEO' : theo_parser
    }

    for entity_type, parser_or_list in all_parsers.items():
        parsers_to_run = parser_or_list if isinstance(parser_or_list, list) else [parser_or_list]

        for parser in parsers_to_run:
            matches = list(parser.findall(text))
            for match in matches:
                span = (match.span.start, match.span.stop)

                if span in predicted_spans:
                    continue

                predicted_spans[span] = entity_type
                entities.append({
                    'text': text[span[0]:span[1]],
                    'type': f"B-{entity_type}"
                })

    return entities

In [64]:
stats = defaultdict(lambda: {'true': 0, 'pred': 0, 'correct': 0})

for tokens, tags in zip(splits['test']['tokens'], splits['test']['ner_tags']):
  text = ' '.join(tokens)
  pred_entities = extract_entities(text)

  true_entities = []
  current_entity, current_type = [], None

  for token, tag in zip(tokens, tags):
      if tag.startswith('B-'):
          tag_type = tag[2:]

          if current_entity and current_type == tag_type:
              current_entity.append(token) # Продолжаем сущность того же типа
          else:
              if current_entity and current_type:
                  true_entities.append({'text': ' '.join(current_entity), 'type': current_type})
              current_entity = [token]
              current_type = tag_type
      else: # O-tag
          if current_entity and current_type:
              true_entities.append({'text': ' '.join(current_entity), 'type': current_type})
          current_entity, current_type = [], None

  if current_entity and current_type:
      true_entities.append({'text': ' '.join(current_entity), 'type': current_type})

  matched_pred_indices = set()

  for entity in true_entities:
      stats[entity['type']]['true'] += 1

  for entity in pred_entities:
      stats[entity['type'][2:]]['pred'] += 1

  for true_ent in true_entities:
      true_text_lower = true_ent['text'].lower()
      true_type = true_ent['type']

      for j in range(len(pred_entities)):
          if j in matched_pred_indices: continue

          pred_ent = pred_entities[j]
          pred_text_lower = pred_ent['text'].lower()
          pred_type = pred_ent['type'][2:]

          is_match = (true_text_lower == pred_text_lower) and (true_type == pred_type)

          if is_match:
              stats[true_type]['correct'] += 1
              matched_pred_indices.add(j)
              break

results = []
all_types = set(stats.keys())
total_true, total_pred, total_correct = 0, 0, 0

for et in sorted(list(all_types)):
  true, pred, correct = stats[et]['true'], stats[et]['pred'], stats[et]['correct']
  total_true += true; total_pred += pred; total_correct += correct

  if true > 0 or pred > 0:
        p = correct / pred if pred > 0 else 0
        r = correct / true if true > 0 else 0
        f1 = 2 * p * r / (p + r) if (p + r) > 0 else 0

        results.append({
            'Entity Type': et, 'Precision': f"{p:.1%}", 'Recall': f"{r:.1%}",
            'F1-Score': f"{f1:.1%}", 'Support': true
        })

p_tot = total_correct / total_pred if total_pred > 0 else 0
r_tot = total_correct / total_true if total_true > 0 else 0
f1_tot = 2 * p_tot * r_tot / (p_tot + r_tot) if (p_tot + r_tot) > 0 else 0

results.append({
  'Entity Type': 'TOTAL', 'Precision': f"{p_tot:.1%}", 'Recall': f"{r_tot:.1%}",
  'F1-Score': f"{f1_tot:.1%}", 'Support': total_true
})


In [65]:
pd.DataFrame(results)

,Entity Type,Precision,Recall,F1-Score,Support
0,FICT,0.0%,0.0%,0.0%,1
1,LOC,16.5%,22.1%,18.9%,86
2,PERSON,0.0%,0.0%,0.0%,151
3,PET,0.0%,0.0%,0.0%,9
4,THEO,0.0%,0.0%,0.0%,4
5,TOTAL,9.9%,7.6%,8.6%,251


Общая Precision 9.9% означает высокий уровень ложных срабатываний => правила слишком общие

Общая Recall 7.6%, т.е. пропускается большинство истинных сущностей в тексте

LOC: Единственная категория с ненулевыми результатами (F1-score 18.9%), но низким Precision 16.5%. Объекты находит, но плохо классифицирует
PERSON, ADDRESS, PET, THEO, FICT, ORG неэффективны

Rule-based показывает низкое качество, сложность в установлении правил на тэги PET, FICT, THEO, поэтому перешли на NN 

In [ ]:
del person_parser,geo_parser,address_parser,org_parser,pet_parser,fict_parser,theo_parser
gc.collect()